In [147]:
import os
import subprocess

programs = ['matrix_multiplication', 'towers_of_hanoi']

In [148]:
def compile_c_to_ir(program, opt_level='-O1'):
    src = f'./c_programs/{program}.c'
    dest = f'{program}.ll'
    
    cmd = ['clang', opt_level, '-S', '-emit-llvm', src, '-o', dest, '-lm', '-w']
    result = subprocess.run(cmd, check=True)

    if result.returncode != 0:
        raise RuntimeError(f'Compilation failed for {program}')
    
    return dest

In [149]:
def apply_flag(program, flags):
    file = f'{program}.ll'

    cmd = ['opt', '-force-remove-attribute=optnone'] + flags + [file, '-S', '-o', file.replace('.ll', '') + '_optimized.ll']
    result = subprocess.run(cmd, check=True)

    if result.returncode != 0:
        raise RuntimeError(f'Applying flags {flags} failed for {program}')
    
    return file

In [150]:
def compile_ir_to_executable(program, is_optimized=True):

    src = f'{program}_optimized.ll' if is_optimized else f'{program}.ll'
    dest = program

    cmd = ['clang', src, '-o', dest, '-lm', '-w']
    result = subprocess.run(cmd, check=True)

    if result.returncode != 0:
        print(f"Clang compilation failed!")
        print(f"Command: {' '.join(cmd)}")
        print(f"stdout: {result.stdout}")
        print(f"stderr: {result.stderr}")
        raise RuntimeError(f'Compilation to executable failed for {program}')

    return dest

In [151]:
def get_static_metrics(ll_file):

    file_size = os.path.getsize(ll_file)
    
    with open(ll_file, 'r') as f:
        content = f.read()
        lines = content.split('\n')
    
    instructions = len([line for line in lines if line.strip().startswith('%') and '=' in line])
    
    functions = len([line for line in lines if line.strip().startswith('define')])
    
    
    basic_blocks = len([line for line in lines if line.strip().endswith(':') and not line.strip().startswith(';')])
    
    load_count = content.count('load')
    store_count = content.count('store')
    call_count = content.count('call')
    
    return {
        'file_size': file_size,
        'total_lines': len(lines),
        'instructions': instructions,
        'functions': functions,
        'basic_blocks': basic_blocks,
        'load_instructions': load_count,
        'store_instructions': store_count,
        'call_instructions': call_count
    }

In [ ]:
import time
import psutil

def get_dynamic_metrics(exec_file):
    try:
        start_time = time.time()
        
        process = subprocess.Popen([f'./{exec_file}'], 
                                 stdout=subprocess.PIPE, 
                                 stderr=subprocess.PIPE)
        
        peak_memory = 0
        timeout_seconds = 30
        
        try:
            ps_process = psutil.Process(process.pid)
            while process.poll() is None:
                elapsed = time.time() - start_time
                if elapsed > timeout_seconds:
                    print(f"Timeout: {exec_file} exceeded {timeout_seconds}s")
                    process.terminate()
                    process.wait()
                    return {'runtime_seconds': timeout_seconds, 'peak_memory_mb': 0}
                    
                try:
                    memory_info = ps_process.memory_info()
                    current_memory = memory_info.rss / (1024 * 1024)
                    peak_memory = max(peak_memory, current_memory)
                    time.sleep(0.01)
                except psutil.NoSuchProcess:
                    break
        except psutil.NoSuchProcess:
            pass
        
        stdout, stderr = process.communicate()
        return_code = process.returncode
        end_time = time.time()
        
        if return_code != 0:
            print(f"{exec_file} failed with return code {return_code}")
            if stderr:
                print(f"Stderr: {stderr.decode()[:200]}")
            return {'runtime_seconds': 0, 'peak_memory_mb': 0}
        
        runtime = round(end_time - start_time, 6)
        
        if runtime < 0.001:
            print(f"{exec_file} runtime too fast: {runtime}s - may indicate failure")
            return {'runtime_seconds': 0, 'peak_memory_mb': 0}
        
        return {
            'runtime_seconds': runtime,
            'peak_memory_mb': round(peak_memory, 2),
        }
        
    except Exception as e:
        print(f"Error measuring {exec_file}: {e}")
        return {'runtime_seconds': 0, 'peak_memory_mb': 0}

In [153]:
def get_metrics(exec_file, is_optimized=True):
    static_metrics = get_static_metrics(exec_file + ('_optimized.ll' if is_optimized else '.ll'))
    dynamic_metrics = get_dynamic_metrics(exec_file)

    complete_metrics = {
        'file_size': static_metrics['file_size'],
        'total_lines': static_metrics['total_lines'],
        'instructions': static_metrics['instructions'],
        'functions': static_metrics['functions'],
        'basic_blocks': static_metrics['basic_blocks'],
        'load_instructions': static_metrics['load_instructions'],
        'store_instructions': static_metrics['store_instructions'],
        'call_instructions': static_metrics['call_instructions'],
        'runtime_seconds': dynamic_metrics['runtime_seconds'],
        'peak_memory_mb': dynamic_metrics['peak_memory_mb']
    }
    
    return complete_metrics

In [154]:
def get_row(program, flags, metrics):
    return [
        program,
        str(flags),
        metrics['file_size'],
        metrics['total_lines'],
        metrics['instructions'],
        metrics['functions'],
        metrics['basic_blocks'],
        metrics['load_instructions'],
        metrics['store_instructions'],
        metrics['call_instructions'],
        metrics['runtime_seconds'],
        metrics['peak_memory_mb']
    ]

In [155]:
def evaluate_program(program, flags):

    data = []

    # No optimization
    compile_c_to_ir(program)
    compile_ir_to_executable(program, is_optimized=False)
    data.append(get_row(program, [], get_metrics(program, is_optimized=False)))

    # Base optimization -O3
    compile_c_to_ir(program, opt_level='-O3')
    compile_ir_to_executable(program, is_optimized=False)
    data.append(get_row(program, ['-O3'], get_metrics(program, is_optimized=False)))
    os.remove(f'{program}.ll')

    compile_c_to_ir(program)

    # Flag sequence
    prefix = []
    for flag in flags:
        prefix.append(flag)
        apply_flag(program, prefix)
        compile_ir_to_executable(program)
        data.append(get_row(program, prefix, get_metrics(program)))

    os.remove(f'{program}.ll')
    os.remove(f'{program}_optimized.ll')
    os.remove(program)

    return data

In [156]:
!pip install tqdm

In [157]:
from itertools import permutations
from tqdm import tqdm
import random

flags = [
    '-inline',      # Function inlining (big changes)
    '-loop-unroll', # Loop unrolling (affects loops)
    '-mem2reg',     # Memory to registers (reduces loads/stores)
    '-instcombine', # Instruction combining (changes instruction counts)
    '-gvn'          # Global value numbering (eliminates redundancy)
]

orders = list(permutations(flags))
# orders = random.sample(orders, 3)

print(f'Total permutations: {len(orders)}')

data = []

total_combinations = len(programs) * len(orders)

with tqdm(total=total_combinations, desc="Generating dataset") as pbar:
    for program in programs:

        # print(f'Processing {program}...')
        for order in orders:
            # Update the progress bar with current info
            pbar.set_description(f"Processing {program}")
            pbar.set_postfix(flags=f"{order}")
            
            data.extend(evaluate_program(program, order))
            
            # Update progress
            pbar.update(1)

        # print(f'Completed {program}.')

Total permutations: 120


Processing matrix_multiplication:  45%|████▍     | 107/240 [21:11<26:05, 11.77s/it, flags=('-gvn', '-loop-unroll', '-instcombine', '-mem2reg', '-inline')]

Timeout: matrix_multiplication exceeded 30s


Processing towers_of_hanoi: 100%|██████████| 240/240 [39:59<00:00, 10.00s/it, flags=('-gvn', '-instcombine', '-mem2reg', '-loop-unroll', '-inline')]          


In [159]:
import pandas as pd

columns = [
    'program',
    'flags',
    'file_size',
    'total_lines',
    'instructions',
    'functions',
    'basic_blocks',
    'load_instructions',
    'store_instructions',
    'call_instructions',
    'runtime_seconds',
    'peak_memory_mb'
]

df = pd.DataFrame(data, columns=columns)
df.to_csv('compiler_optimization_dataset.csv', index=False)

print(df.head(10))
print(f'Total rows in dataset: {len(df)}')

                 program                                              flags  \
0  matrix_multiplication                                                 []   
1  matrix_multiplication                                            ['-O3']   
2  matrix_multiplication                                        ['-inline']   
3  matrix_multiplication                        ['-inline', '-loop-unroll']   
4  matrix_multiplication            ['-inline', '-loop-unroll', '-mem2reg']   
5  matrix_multiplication  ['-inline', '-loop-unroll', '-mem2reg', '-inst...   
6  matrix_multiplication  ['-inline', '-loop-unroll', '-mem2reg', '-inst...   
7  matrix_multiplication                                                 []   
8  matrix_multiplication                                            ['-O3']   
9  matrix_multiplication                                        ['-inline']   

   file_size  total_lines  instructions  functions  basic_blocks  \
0      23189          573           274          8            